# Analysis of Ar41 export by 2M Neutrons
In this notebook, we see the exportation of Ar41 events selected by the 2M neutrons simulation.

Please note that we convert the energies **in KeV** to be consistent with the previous studies.

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [25, 15]
plt.rcParams.update({'font.size': 32})

In [ ]:
def load_dataframe_from_files(dirin, fileprefix, max_files=100):
    import glob
    files = glob.glob(os.path.join(dirin, fileprefix))
    print("[Info] Loading {} files wt prefix:\n{}".format(len(files), fileprefix))
    df = pd.read_csv(files[0], comment='#', index_col=False)
    for file in files[1:max_files]:
        print(".", end='')
        dftmp = pd.read_csv(file, comment='#', index_col=False)
        df = pd.concat([df, dftmp])
    print("")
    return df

In [ ]:
df = load_dataframe_from_files(".", "ExportAr41*csv")

In [ ]:
print("[Info] Loaded {} events wt Ar41".format(df.eventnumber.nunique()))

In [ ]:
edep_gby_event = df.groupby("eventnumber").energydeposition.sum()
edeplar_gby_event = df[df.SensitiveVolID==201].groupby("eventnumber").energydeposition.sum()
edepfib_gby_event = df[df.SensitiveVolID!=201].groupby("eventnumber").energydeposition.sum()

In [ ]:
bins = np.linspace(0, edep_gby_event.max(), 100)
plt.hist([edeplar_gby_event, edepfib_gby_event], bins=bins, label=["Deposit in LAr", "Deposit on FiberCores"], alpha=0.75, edgecolor='w', stacked=True)
plt.xlabel("Energy deposition per event (MeV)")
plt.ylabel("Count(#events)")
plt.title("Energy deposition - 2M Neutrons (Only events wt Ar41)")
plt.text(100, 1500, "Only LAr:\nMean: {:.2f} KeV, Max: {:.2f} KeV".format(edeplar_gby_event.mean(), edeplar_gby_event.max()))
plt.text(100, 1200, "Only FiberCores:\nMean: {:.2f} KeV, Max: {:.2f} KeV".format(edepfib_gby_event.mean(), edepfib_gby_event.max()))
plt.text(100, 900, "Almost all the deposit are in LAr.\nDo we need to enable FiberCore? (34GB data)")
plt.legend()
plt.show()

# Conclusions:
1. Do we really need to simulate the fiber core? Maybe we can save resources by recording only LAr entries.

In [ ]:
first_gammas = df[(df.PID==22) & (df.creatorprocess=="nCapture")].groupby(["eventnumber", "tracknumber"]).first().reset_index()

In [ ]:
cum_ke_first_gammas = first_gammas.groupby("eventnumber").kineticenergy.sum()

In [ ]:
plt.hist(cum_ke_first_gammas, bins=500, edgecolor='k', color='w')
plt.title("Energy of Gammas from Ar41 de-excitation")
plt.xlabel("energy (KeV)")
plt.ylabel("count (#events)")
plt.show()

In [ ]:
ar41_creationtime = df[df.PID==1000180410].groupby(["eventnumber", "tracknumber"]).first().reset_index().time

In [ ]:
bins = np.linspace(0, ar41_creationtime.max(), 1000)
plt.hist(ar41_creationtime, bins=bins, density=True)
plt.yscale("log")
plt.show()

In [ ]:
ar41_creationtime.describe()

In [ ]:
len(ar41_creationtime[ar41_creationtime<271*1000])/len(ar41_creationtime)